<a target="_blank" href="https://colab.research.google.com/github/upadhyan/SUDS-2024-Bootcamp-Friday/blob/main/Supervised_Learning_Lab.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# UofT DSI SUDS
#### Supervised Learning Lab
#### Teaching team: Nakul Upadhya
##### Lab author: Kyle E. C. Booth, kbooth@mie.utoronto.ca, edited by Jake Mosseri, Nakul Upadhya, Eldan Cohen, Alex Olson, Shehnaz Islam, and Faisal Shaik

---
#### Note from Faisal (mrdandelion6):
I copied this file from a different repository as part of my training in data science at SUDS UofT 2024. Most of the content in these files is not written by me, I have just made some extra edits. Credit to the original authors and editors. You can see the original repository here: https://github.com/upadhyan/SUDS-2024-Bootcamp-Friday

---

In this lab, we will be introducing *decision tree and forests*. We will introduce the notion of a decision tree, extend this to random forests, and then investigate some state-of-the-art tree-based methods for machine learning. Let's get started!

In [3]:
import pandas as pd
import numpy as np
import sklearn

### Decision Trees

Decision trees are popular supervised learning methods used for classification and regression. The tree represents a series of simple decision rules that predict the target when the feature vector is passed through them. Decision trees are easy to understand, can be visualized nicely, require very little data preparation (e.g., we don't need to scale features), and the trained model can be explained easily to others post priori (as opposed to other *black box* methods that are difficult to communicate).

###### Example
Suppose you wanted to design a simple decision tree for whether (or not) you buy a used car. You might develop something like the following:

<img src="https://github.com/lyeskhalil/mlbootcamp/blob/master/img/decision-tree.gif?raw=1" width="500"/>

**YOUR TURN:** Let's say you're browsing Kijiji and come across a used car that: has been road tested, has high mileage, and is a recent year/model.
* According to your decision tree model, should you buy this car or not? ____________________________
* Will you buy any cars that haven't been road tested (if you follow your model)? ___________________________________

Obviously this tree may not be ideal, depending on the situation. For example, you could have a road tested car of a recent year with 2,000,000 km's on it and the model is telling you to buy! (But, you probably shouldn't)



**Titanic Survivor Dataset**

In this notebook, we will be exploring the use of decision trees in the context of Kaggle's famous **Titanic dataset**. Each row in the data represents a passenger, detailing various characteristics about them (i.e., the features), and also details whether or not the passenger survived the disaster.

Let's load the data and take a look at it.

To get the data into a manageable format, we're going to use the [Pandas](https://pandas.pydata.org/) library, a popular library for data manipulation and analysis. For information on Pandas, see [`data_analysis.ipynb`](../data_analysis.ipynb).

In [4]:
import pandas as pd # import pandas to get access to dataframe operations
from sklearn.datasets import fetch_openml # import function to retrieve relevant datasets

full_data = fetch_openml("titanic", version=1, as_frame=True) # Get all data and metadata
data = full_data.frame # Extract the relevant data
data.survived = pd.to_numeric(data['survived'])
data.drop(['boat', 'body', 'home.dest'], axis=1, inplace=True) # Drop irrelevant columns
data.head() # view the first 5 rows

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S


The above cell used the `fetch_openml` function to pull in the Titanic survivor data from the OpenML website. The `.head()` allows us to conveniently take a glance at the first 5 rows (along with the header). We set `as_frame=True` so fetch_openml returns the data as a Pandas data frame. If `as_frame` is set to False or not specified, the data will be returned as a NumPy `ndarray`.

`fetch_openml` will actually return it as a `Bunch` object directly, and you can then pull the data frame from the `.frame` attribute. Hence our `data` variable is a Pandas data frame.

We can see that, along with the target 'Survived', we have a number of features including the passenger name, sex, age, fare, cabin, etc. We can do a bit of simple *exploratory data analysis* (EDA) to get a better feel for the dataset.

In [5]:
print ("Passengers, features: ", data.shape)
print ("Survived: ", data[data["survived"]==1].shape[0], ", Didn't: ", data[data["survived"]==0].shape[0])
# explanation of data[data["survived"]==1].shape[0] syntax:
# we use pandas DataFrame filterring to count the number of rows where the "survived" column equalls to 1.

print ("female: ", data[data["sex"]=="female"].shape[0], ", Male: ", data[data["sex"]=="male"].shape[0])
print ("\n Missing values by feature: \n", data.isna().sum())

Passengers, features:  (1309, 11)
Survived:  500 , Didn't:  809
female:  466 , Male:  843

 Missing values by feature: 
 pclass         0
survived       0
name           0
sex            0
age          263
sibsp          0
parch          0
ticket         0
fare           1
cabin       1014
embarked       2
dtype: int64


As you can see, we can use Pandas to conveniently summarize key aspects of the dataset such as the number of passengers, features, survived/didn't, and their gender. We are also able to identify the number of missing values per feature in the dataset.

To accomplish this, we used Pandas flexible indexing capability. The syntax `data[data[col]==val]` allows us to return the subset of rows in `data` where column `col` takes on value `val`. Very powerful!

As you may have suspected, the dataset we're using is actually a subset of the total Titanic data. In reality, there were actually 3,547 passengers while the data we're working with only concerns 1309 of them.

**YOUR TURN:**
Using similar syntax, answer the following questions about the data:
* In the dataset, what is the passenger survival rate? ____________________________
* How many passengers paid more than $10 for fare? ____________________________
* How many passengers had a passenger class (Pclass) of 3? ________________________
* With some discussion/exploration and try to determine what features might be the most relevant to passenger survival.

In [ ]:
## Your code here
##
##

##### Data Preparation: Categorical -> Numerical Mapping

Before we can fit sklearn decision trees to our data, we first need to convert all of the categorical variables (e.g., gender) numerical values - this is called *encoding*. In previous labs, we dealt with datasets that were pre-prepared; now things are getting a little more realistic! Categoricals with unique values (like name and ticket #) can be removed from the dataset entirely as we don't suspect they will contribute to the model.

We can do the required preparation as follows:

In [9]:
from sklearn import preprocessing

data = data.drop(['name', 'ticket', 'cabin', 'embarked'], axis=1) # remove unimportant columns

le = preprocessing.LabelEncoder() # Create a label encoder
le.fit(data['sex']) # provide data for it to learn what classes there are
data['sex'] = le.transform(data['sex']) # apply the encoding

data.head()

,pclass,survived,sex,age,sibsp,parch,fare
0,1,1,0,29.0000,0,0,211.3375
1,1,1,1,0.9167,1,2,151.5500
2,1,0,0,2.0000,1,2,151.5500
3,1,0,1,30.0000,1,2,151.5500
4,1,0,0,25.0000,1,2,151.5500


In the above cell, we dropped a number of columns we don't suspect will be correlated with the target (*Note: we probably should have been a bit more rigorous about this!*). Then we used the `LabelEncoder()` within sklearn that can fit a numbering scheme to a categorical feature (i.e., 'Sex'). We can see in the new dataset, sex takes on a value of 0 (female) or 1 (male).

##### Model Development

OK! Let's get to developing some decision tree models to predict passenger survival. We will start with simple decision trees and develop more complex models from there. Our first step, as in previous labs, is to split our data into a training set and a test set (unseen data). We will then use k-folds cross validation on the training set to try and get the best performing model before finally applying it to the test data.

Let's import sklearn's decision tree classifer and split the data (using techniques we covered in the first lab).

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree # import our decision tree model

target_data = data["survived"]
feature_data = data.iloc[:, data.columns != "survived"]

X_train, X_test, y_train, y_test = train_test_split(feature_data, target_data, test_size=0.3, random_state=0)

Note that `random_state` ensures stability by using the same random seed each time.

**YOUR TURN:**
* How many samples are in the training set? _______________________
* How many samples are in the test set? _______________________
* What are the survival rates in each of the datasets? ______________________

In [ ]:
## Your code here
##
##
##

##### Dealing with Missing Data: Imputation

Before we can fit our decision tree to our training data, we can conduct *imputation* to replace missing values with the mean/median/mode value in the column. For this exercise we will conduct mode imputation (i.e., the most common value in the column).

**YOUR TURN:** Assuming we have a feature vector with three rows where 'nan' is a missing value:
X = [[1, 2, 3],
     [1, 2, nan],
     [2, 3, 2]]
* Which sample has a missing value? (the 1st, 2nd or 3rd?) ______________________________
* If we *impute* (i.e., replace the missing value with another value) using the mean (average), what value will go in place of the nan value? __________________________________

It's important that you don't impute your data using statistics including the the test data! This is an example of *information leak* where your test data is leaking into your training data.

As such, we will fit our missing data imputer to our training data only.

In [7]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

imp.fit(X_train)
X_train = imp.transform(X_train) # replace missing data using our imputer

##### Fitting a Tree

So we've got our data prepared, let's fit a decision tree to our training data.

Remember, the pipeline for model development in sklearn is **initialize->fit->predict**.

In [10]:
from sklearn.metrics import accuracy_score
from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth = 11)

clf.fit(X_train, y_train)

accuracy = accuracy_score(y_train, clf.predict(X_train))
print ("Accuracy: ", accuracy * 100, "%")

ValueError: could not convert string to float: 'Mellinger, Miss. Madeleine Violet'

In the above cell, we defined a Decision Tree classifier and fit it to our training set.

**YOUR TURN:**
* What is the performance of this model on the test set?
* Do you think the model has overfit or underfit: ____________


In [ ]:
## Your code here
##
##
##

##### Varying Depth

Now that we know how to fit a tree, lets see what impact the depth of a tree has on its performance

**Your Turn**
* Vary the depth of the tree and calculate its train and test accuracy at each depth (Hint: control the maximumum depth using the `max_depth` parameter
* Plot the train and test accuracy together.
* At what depth is the model underfitting and at what depth is the model overfitting? _______________


In [ ]:
import matplotlib.pyplot as plt



## Your training code here
##
##
##


fig, ax = plt.subplots()
## Your plotting code here
##
##
##
ax.set_title('Model Performance across Depth')
fig.show()


##### Visualizing the Tree

One useful thing we can do is actually visualize our decision tree model! We can use the [graphViz](https://www.graphviz.org/) library to accomplish this:

In [ ]:
from sklearn.tree import export_graphviz
import graphviz # Package containing visualization tools

clf = tree.DecisionTreeClassifier(max_depth = 3)

clf.fit(X_train, y_train)

accuracy = accuracy_score(y_train, clf.predict(X_train))
print ("Accuracy: ", accuracy * 100, "%")

export_graphviz(clf, out_file="mytree.dot", feature_names=data.columns.drop('survived')) # Save the visualization of the tree
with open("mytree.dot") as f: # read the file back in
    dot_graph = f.read()
graphviz.Source(dot_graph) # display the tree

**YOUR TURN:** Explore the decision tree and answer the following:
* What feature does the root node split on?
* What features appear most in the tree?

##### HP Tuning
There are many hyper-parameters that can be tuned to change how the model performs. Some common parameters that are modified include:
1. Max Tree Depth: How "tall" do you want your tree to be
2. Minimum Samples Per Leaf: This parameter defines the minimum number of training datapoints that fall into a given leaf node in order for that node to be created
3. Minimum Samples to Split: This parameter controls the minimum number of samples required to create a decision split

To decide the values of each of the parameters, we can use Grid Search combined with cross validation. In Grid Search, we first decide what potential values we want each hyperparameter will take. Then we find every possible combination of parameters and run cross validation on each combination to estimate the performance of that hyperparameter combination.

Luckily, `sklearn` has a nice implementation of Grid Search that runs this algorithm for us.

**Your Turn**

Here we want to tune three parameters: max_depth, min_samples_split, and min_samples_leaf. To do this, we need to define possible values we want to search over.
* In the cell below, define possible values. NOTE: the more values you specify, the longer grid search will take.


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

clf = tree.DecisionTreeClassifier() # First we define our model without passing in parameters
hyperparameter_search = { # Then we decide the possible parameter combinations
    'max_depth': [], ## FILL THIS IN
    'min_samples_split': [2, 5, 8],
    'min_samples_leaf': [] ## FILL THIS IN
}
evaluation_metric = make_scorer(accuracy_score, # GridSearchCV requires us to wrap our metric function in a "scorer"
                                greater_is_better = True)

grid_search_cv = GridSearchCV(estimator = clf,
                              param_grid = hyperparameter_search,
                              scoring = evaluation_metric,
                              cv = 5) # Set up search algorithm
grid_search_cv.fit(X_train, y_train) # Run the search. NOTE: This may take a while

print("Best Parameters: ", grid_search_cv.best_params_) # Print the parameters
print ("Best CV Accuracy: ", grid_search_cv.best_score_ * 100, "%")

clf = grid_search_cv.best_estimator_ # Get the best model from the GridSearch
accuracy = accuracy_score(y_test, clf.predict(imp.transform(X_test)))
print ("Testing Accuracy: ", accuracy * 100, "%") # Print the testing accuracy of the best model

In the cell above, we tested our three values per hyperparameter and ran grid search to find the best combination from the space we defined. As you may have noticed, the number of combinations tested by Grid Search exponentially increases as you test more values and tune more hyperparameters. This means that performing a grid search is often a task that takes a long period of time and is often **not** used for more complex models like neural networks.

### Other Models
If you have finished early, feel free to try other models and try to get as high of a test accuracy as you can!

Some other models you can start with:
1. [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
2. [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)
3. [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

You can also use models from other packages. Some examples include:
1. [XGBoost](https://xgboost.readthedocs.io/en/stable/get_started.html) (you will need to run `!pip install xgboost`)
2. [LightGBM](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html#lightgbm.LGBMClassifier) (`!pip install lightgbm`)

